In [1]:
# Music processing
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Dataframe
import pandas as pd

### Credentials

In [2]:
CLIENT_ID = "7dbe81718b764f1093de228de9f29874"
CLIENT_SECRET = "7f31515feefd44559d7e9d8a869c3e99"

### Access Spotify API

In [3]:
token = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

c:\users\rmfer\desktop\ufm\semestre 7\machine learning models\virtual_64\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


## Get all tracks of a playlist 

In [4]:
# Get all tracks of a playlist
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

## Create a Dataframe of the playlist

In [5]:
def analyze_playlist(playlist_tracks):
    
    # Create empty dataframe
    playlist_features_list = ["artist","album","track_name",  "track_id","acousticness","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    for track in playlist_tracks:
        # Create empty dict
        playlist_features = {}
        
        try:
            # Get metadata
            playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
            playlist_features["album"] = track["track"]["album"]["name"]
            playlist_features["track_name"] = track["track"]["name"]
            playlist_features["track_id"] = track["track"]["id"]

            # Get audio features
            audio_features = sp.audio_features(playlist_features["track_id"])[0]
            for feature in playlist_features_list[4:]:
                playlist_features[feature] = audio_features[feature]

            # Concat the dfs
            track_df = pd.DataFrame(playlist_features, index = [0])
            playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        except:
            continue
        
    return playlist_df

## Multiple Playlist generator

In [6]:
def analyze_playlist_dict(playlist_dict):
    
    # Loop through every playlist in the dict and analyze it
    for i, (key, val) in enumerate(playlist_dict.items()):
        #playlist_df = analyze_playlist(*val)
        
        temp = get_playlist_tracks(*val)
        playlist_df = analyze_playlist(temp)
        
        # Add a playlist column so that we can see which playlist a track belongs too
        playlist_df["playlist"] = key
        
        # Create or concat df
        if i == 0:
            playlist_dict_df = playlist_df
        else:
            playlist_dict_df = pd.concat([playlist_dict_df, playlist_df], ignore_index = True)
            
    return playlist_dict_df

## Getting and transforming data

In [7]:
playlist_dict = {
    'Varias (80s,90s...)': ('Fernando Gonzalez', '21BtvJNHCJJARHFq8VROUM'), 
     #'Pop' : ('redmusiccompany', '64QxysD2w5x5EMLgcoT7fa'),
     'Salsa' : ('Angelicaptg', '5XsWICVzVs7U3FldA4rD3x'),
     'Electronic' : ('Juho Raivola', '6aiEWJB3gfGBudAZr5wDVt'),
     #'Instrumental' : ('Keats Handwriting', '21J20PgNUj49fSp2UOfawo')
    
}

In [8]:
music_df = analyze_playlist_dict(playlist_dict)

In [9]:
music_df["playlist"].value_counts()

Varias (80s,90s...)    573
Salsa                  564
Electronic             505
Name: playlist, dtype: int64

In [10]:
music_df.head()

,artist,album,track_name,track_id,acousticness,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
0,TOTO,Toto IV,Africa,2374M0fQpWi3dLnB54qaLX,0.257000,0.671,0.373,9,-18.064,1,0.0323,0.000079,0.0481,0.732,92.717,295893,4,"Varias (80s,90s...)"
1,TOTO,Toto,Hold the Line,4aVuWgvD0X63hcOCnZtNFA,0.007830,0.473,0.898,6,-4.797,0,0.0561,0.061800,0.1730,0.808,96.556,235547,4,"Varias (80s,90s...)"
2,TOTO,Toto IV,Rosanna,37BTh5g05cxBIRYMbw8g2T,0.021300,0.384,0.512,7,-13.035,0,0.0358,0.000139,0.2050,0.739,78.442,331200,4,"Varias (80s,90s...)"
3,U2,The Best of 1980-1990 & B-Sides,With Or Without You,2JZfTvWWtpaE8NohqRXqFr,0.000128,0.546,0.485,2,-9.424,1,0.0270,0.318000,0.0939,0.106,109.913,295200,4,"Varias (80s,90s...)"
4,U2,U218 Singles (Deluxe Version),Beautiful Day,0G5F2msfVO77xs7ql2RiTS,0.022500,0.541,0.914,2,-6.727,1,0.0520,0.000441,0.3530,0.449,136.282,245707,4,"Varias (80s,90s...)"


In [11]:
music_df.tail()

,artist,album,track_name,track_id,acousticness,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
1637,Kerri Chandler,Bar A Thym (Supernova Remix Deluxe),Bar A Thym (Original),5sEsABi0eX7MxBQR0uYKaT,0.00045,0.806,0.869,10,-4.553,0,0.0570,0.9080,0.0570,0.7580,125.993,444624,4,Electronic
1638,Jean-Michel Jarre,Oxygène,"Oxygene, Pt. 4",0Tt2jli0yAD1MKSTb5MvS2,0.02340,0.430,0.755,7,-8.457,0,0.0365,0.5910,0.5510,0.6570,188.904,246893,3,Electronic
1639,FC Kahuna,Machine Says Yes,Hayling (feat. Hafdis Huld),58eKLwmiYrJAqNlv96pwlP,0.49500,0.555,0.329,5,-9.188,1,0.0279,0.0154,0.6690,0.0391,140.038,407973,4,Electronic
1640,Hollis P. Monroe,I'M Lonely,I'm Lonely,7iEgoNG4xlbpTxLuMNrBl5,0.00102,0.656,0.864,7,-6.423,1,0.0444,0.7870,0.1580,0.8150,133.305,413120,4,Electronic
1641,DJ Q,Twentyfourseven,Optiumum Thinking,50ti9tKpZpgmT3BCe4Ezi0,0.04850,0.838,0.733,6,-10.467,1,0.0308,0.8670,0.0825,0.5450,127.889,457960,4,Electronic


In [12]:
# csv
music_df.to_csv("./Dataframe/MusicDataNo.1.csv", index = False)

In [13]:
music_df.head()

,artist,album,track_name,track_id,acousticness,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
0,TOTO,Toto IV,Africa,2374M0fQpWi3dLnB54qaLX,0.257000,0.671,0.373,9,-18.064,1,0.0323,0.000079,0.0481,0.732,92.717,295893,4,"Varias (80s,90s...)"
1,TOTO,Toto,Hold the Line,4aVuWgvD0X63hcOCnZtNFA,0.007830,0.473,0.898,6,-4.797,0,0.0561,0.061800,0.1730,0.808,96.556,235547,4,"Varias (80s,90s...)"
2,TOTO,Toto IV,Rosanna,37BTh5g05cxBIRYMbw8g2T,0.021300,0.384,0.512,7,-13.035,0,0.0358,0.000139,0.2050,0.739,78.442,331200,4,"Varias (80s,90s...)"
3,U2,The Best of 1980-1990 & B-Sides,With Or Without You,2JZfTvWWtpaE8NohqRXqFr,0.000128,0.546,0.485,2,-9.424,1,0.0270,0.318000,0.0939,0.106,109.913,295200,4,"Varias (80s,90s...)"
4,U2,U218 Singles (Deluxe Version),Beautiful Day,0G5F2msfVO77xs7ql2RiTS,0.022500,0.541,0.914,2,-6.727,1,0.0520,0.000441,0.3530,0.449,136.282,245707,4,"Varias (80s,90s...)"
